In [88]:
import io
import re
import itertools

import netCDF4
import pandas
import numpy as np

import jinja2

In [45]:
ds = netCDF4.Dataset('../data/transect.nc')

In [46]:
variables = {
    'id': {"var": 'id', "slice": np.s_[:]},
    'lat_0': {"var": 'lat', "slice": np.s_[:, 0]},
    'lat_1': {"var": 'lat', "slice": np.s_[:, -1]},
    'lon_0': {"var": 'lon', "slice": np.s_[:, 0]},
    'lon_1': {"var": 'lon', "slice": np.s_[:, -1]},
    'rsp_lon': {"var": 'rsp_lon', "slice": np.s_[:]},
    'rsp_lat': {"var": 'rsp_lat', "slice": np.s_[:]}
}
data = {}
for var, props in variables.items():
    data[var] = ds.variables[props['var']][props['slice']]
df = pandas.DataFrame(data=data)
df['north'] = df['rsp_lat'] + 0.002
df['south'] = df['rsp_lat'] - 0.002
df['east'] = df['rsp_lon'] + .0025
df['west'] = df['rsp_lon'] - .0025


In [76]:
def textcoordinates(x0, y0, z0=None, x1=None, y1=None, z1=None):
    """
    convert the coordinates to a string so they can be used by kml

    # Example usage:
    >>> x = np.array([1,1,1])
    >>> y = np.array([1,-2e10,3.0000001])
    >>> print(textcoordinates(x, y))
    1.0,1.0,0.0
    1.0,-20000000000.0,0.0
    1.0,3.0000001,0.0
    <BLANKLINE>
    >>> # Allow for coordinate pairs
    >>> x0 = x
    >>> y0 = y
    >>> x1 = x
    >>> y1 = y
    >>> print(textcoordinates(x0=x0, y0=y0, x1=x1, y1=y1))
    1.0,1.0,0.0 1.0,1.0,0.0
    1.0,-20000000000.0,0.0 1.0,-20000000000.0,0.0
    1.0,3.0000001,0.0 1.0,3.0000001,0.0
    <BLANKLINE>
    >>> x = np.array([1])
    >>> y = np.array([1])
    >>> z = np.array([np.nan])
    >>> print(textcoordinates(x, y, z))
    <BLANKLINE>

    """
    if z0 is None:
        z0 = np.zeros_like(x0)
    if x1 is None:
        coordinates = np.vstack([x0, y0, z0]).T
    else:
        if z1 is None:
            z1 = np.zeros_like(x1)
        coordinates = np.vstack([x0, y0, z0, x1, y1, z1]).T
    # only write coordinates where none is nan
    filter = np.isnan(coordinates).any(1)
    # use cStringIO for performance
    output = io.BytesIO()
    # save coordinates to string buffer
    # I think this is faster than other methods
    # Use %s to get reduced string output
    np.savetxt(output, coordinates[~filter], delimiter=',', fmt='%s')
    coord_bytes = output.getvalue()
    coord_str = coord_bytes.decode()
    if x1 is not None:
        # replace all the 3rd , by a space...
        # TODO double check this regex
        coord_str = re.sub(r'(.*?,.*?,.*?),(.*)',  r'\1 \2', coord_str)
    return coord_str


In [77]:
def line_coords(record):
    return textcoordinates(
        x0=record.lon_0, 
        y0=record.lat_0, 
        x1=record.lon_1, 
        y1=record.lat_1
    )
def point_coords(record):
    return textcoordinates(
        x0=record.rsp_lat,
        y0=record.rsp_lon
    )
def bbox(record):
    return {
        'north': record.north,
        'south': record.south,
        'east': record.east,
        'west': record.west
    }

In [93]:
df['line_coords'] = df.apply(line_coords, axis=1)
df['point_coords'] = df.apply(point_coords, axis=1)
df['bbox'] = df.apply(bbox, axis=1)
n_pixels = itertools.islice(
    itertools.cycle([64, 32, 64, 16, 64, 32, 64, 8]),
    len(df)
)
df['min_lod_pixels'] = list(n_pixels)

In [94]:
df.head()

,id,lat_0,lat_1,lon_0,lon_1,rsp_lat,rsp_lon,north,south,east,west,line_coords,point_coords,bbox,min_lod_pixels
0,2000100,53.490362,53.404006,6.146822,6.140752,53.471422,6.145488,53.473422,53.469422,6.147988,6.142988,"6.1468218159,53.4903622618,0.0 6.14075185913,5...","53.4714215203,6.14548829642,0.0\n","{'south': 53.46942152032584, 'west': 6.1429882...",64
1,2000101,53.490005,53.405276,6.151774,6.123162,53.471422,6.145488,53.473422,53.469422,6.147988,6.142988,"6.15177363461,53.4900045029,0.0 6.12316167857,...","53.4714215203,6.14548829642,0.0\n","{'south': 53.46942152032584, 'west': 6.1429882...",32
2,2000102,53.489189,53.408174,6.156570,6.106119,53.471422,6.145488,53.473422,53.469422,6.147988,6.142988,"6.15657048638,53.489188978,0.0 6.10611872132,5...","53.4714215203,6.14548829642,0.0\n","{'south': 53.46942152032584, 'west': 6.1429882...",64
3,2000103,53.487936,53.412626,6.161094,6.090041,53.471422,6.145488,53.473422,53.469422,6.147988,6.142988,"6.16109412018,53.4879357932,0.0 6.09004091911,...","53.4714215203,6.14548829642,0.0\n","{'south': 53.46942152032584, 'west': 6.1429882...",16
4,2000104,53.486459,53.417873,6.164839,6.076723,53.471422,6.145488,53.473422,53.469422,6.147988,6.142988,"6.16483945767,53.4864590501,0.0 6.07672286469,...","53.4714215203,6.14548829642,0.0\n","{'south': 53.46942152032584, 'west': 6.1429882...",64


In [109]:
template = """
<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">
    <Document>
        <name>Transect Lod</name>
        <open>1</open>
        <Style id="thin">
            <LineStyle>
                <color>33ffffff</color>
            </LineStyle>
            <LabelStyle>
                <scale>0</scale>
            </LabelStyle>
        </Style>
        <Style id="thick">
            <IconStyle>
                <scale>5</scale>
            </IconStyle>
            <LineStyle>
                <color>88ffffff</color>
            </LineStyle>
            <LabelStyle>
                <scale>1.1</scale>
            </LabelStyle>
        </Style>
        <StyleMap id="whiteline">
            <Pair>
                <key>normal</key>
                <styleUrl>#thin</styleUrl>
            </Pair>
            <Pair>
                <key>highlight</key>
                <styleUrl>#thick</styleUrl>
            </Pair>
        </StyleMap>
        <Folder>
            {% for id, line in lines.iterrows() %}
            <NetworkLink>
                <name>Transect {{ line.id }}</name>
                <Region>
                    <LatLonAltBox>
                        <north>{{ line.bbox.north }}</north>
                        <south>{{ line.bbox.south }}</south>
                        <east>{{ line.bbox.east }}</east>
                        <west>{{ line.bbox.west }}</west>
                        <minAltitude>-20</minAltitude>
                        {# Make 50 meters high, so you get more transects if you fly over them #}
                        <maxAltitude>50</maxAltitude>
                        <altitudeMode>absolute</altitudeMode>
                    </LatLonAltBox>
                    <Lod>
                        {# Don't show too many lines #}
                        {# alternate lod, so we get different transects at different levels #}
                        <minLodPixels>{{ line.min_lod_pixels }}</minLodPixels>
                    </Lod>
                </Region>
                <Link>
                    {# 
                    Replace these query parameters in the url tag... (or extend url to match these parameters)
                    Don't forget to escape &.
                    #}
                    <href>/transect/{{line.id}}/kml</href>
                    <viewRefreshMode>onRegion</viewRefreshMode>
                </Link>
            </NetworkLink>
            <Placemark id="{{ line.id }}">
                <name>Transect {{ line.id }}</name>
                <styleUrl>#whiteline</styleUrl>
                <LineString>
                    <coordinates>{{ line.coordinates }}</coordinates>
                </LineString>
                <description>
                    <![CDATA[
                    <div>
                        <a href="/transect/{{ line.id }}/info" data-dynamic-info="true">info</a>
                    </div>
                    ]]>
                </description>
            </Placemark>
            {% endfor %}
        </Folder>
        {# ensures the style KML is preloaded, so there's no delay when all transects load #}
        <Placemark id="style-preloader">
            <name>Style preloader</name>
            <styleUrl>/transect/style</styleUrl>
            <LineString>
                <coordinates>0,0,0 0.1,0.1,0.1</coordinates>
            </LineString>
        </Placemark>
    </Document>
</kml>
"""

In [113]:
kml = jinja2.Template(template).render(lines=df)

565 ms ± 8.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
